In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [8]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

# Create Torch Dataset

In [14]:
class KeypointsData(Dataset):
    def __init__(self, img_dir, datafile):
        self.img_dir = img_dir
        with open(datafile, "r") as f:
            self.data = json.load(f)
            
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        items = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        ht, wth = img.shape[:2]
        
        img = cv2.cvtColor(img, cv2.COLORBGR2RGB)
        img = self.transforms(img)
        keypts = np.array(items['keypts']).flatten()
        keypts = keypts.astype(np.float32)
        
        keypts[::2] *= 224 / wth
        keypts[1::2] *= 224 / ht
        
        return img,keypts

In [16]:
train_data = KeypointsData("data/images", "data/data_train.json")
val_data = KeypointsData("data/images", "data/data_val.json")

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
val_loader = DataLoader(val_data, batch_size=8, shuffle=True)

# Create Model

In [17]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2)

/Users/nicholasmoy/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/nicholasmoy/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/nicholasmoy/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|███████████████████████████████████████████████████████████| 97.8M/97.8M [00:41<00:00, 2.45MB/s]


In [18]:
model = model.to(device)

# Train Model

In [19]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
epochs=20
for epoch in range(epochs):
    for i, (img, keypts) in enumerate(train_loader):
        imgs = imgs.to(device)
        keypts = keypts.to(device)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, keypts)
        loss.backward()
        optimizer.step()
        
        if i % 10 == 0:
            print(f"Epoch: {epoch}, iter: {i}, loss: {loss.item()}")